In [1]:
from AOC_utils import get_day
import numpy as np

day = 21
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 21 input already downloaded
5


['480A', '143A', '983A', '382A', '974A']

In [2]:
example = '''
029A
980A
179A
456A
379A
'''.split('\n')[1:-1]

In [3]:
def solve_day(data):
    part1 = 0
    part2 = 0

    final_codes = data

    numpad = np.array([['7', '8', '9'], 
                       ['4', '5', '6'], 
                       ['1', '2', '3'],
                       [' ', '0', 'A']])
    
    robot_pad = np.array([[' ', '^', 'A'],
                         ['<', 'v', '>']])
    
    numpad_mapping = {}
    for current_n in numpad.flatten():
        for target_n in numpad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            distance = np.argwhere(numpad == current_n)[0] - np.argwhere(numpad == target_n)[0]
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            if current_n == 'A' or current_n == '0':
                move = (up * '^') + (down * 'v') + (left * '<') + (right * '>')
            else:
                move = (left * '<') + (right * '>') + (up * '^') + (down * 'v')

            numpad_mapping[current_n + target_n] = move

    robot_mapping = {}
    for current_n in robot_pad.flatten():
        for target_n in robot_pad.flatten():
            if current_n == ' ' or target_n == ' ':
                continue

            distance = np.argwhere(robot_pad == current_n)[0] - np.argwhere(robot_pad == target_n)[0]
            up = distance[0] if distance[0] > 0 else 0
            down = abs(distance[0]) if distance[0] < 0 else 0
            left = distance[1] if distance[1] > 0 else 0
            right = abs(distance[1]) if distance[1] < 0 else 0

            if current_n == '<':
                move = (left * '<') + (right * '>') + (up * '^') + (down * 'v')
            else:
                move = (up * '^') + (down * 'v') + (left * '<') + (right * '>')

            robot_mapping[current_n + target_n] = move

    def fcode_to_robot(code, current='A'):
        move = ''
        for c in code:
            move += numpad_mapping[current + c] + 'A'
            current = c
        return move
    
    def robot_to_robot(code, current='A'):
        move = ''
        for c in code:
            move += robot_mapping[current + c] + 'A'
            current = c
        return move
    

    for code in final_codes:
        print(code)
        numpad_code = fcode_to_robot(code)
        print(numpad_code)
        robot_code = robot_to_robot(numpad_code)
        print(robot_code)
        robot_code2 = robot_to_robot(robot_code)
        print(robot_code2)
        print(len(robot_code2), int(code[:-1]))
        part1 += len(robot_code2) * int(code[:-1])

    print("part 1:", part1)
    print("part 2:", part2)

solve_day(example)

029A
<A^A>^^AvvvA
v<<A>>^A<A>AvA^<AA>Av<AAA^>A
v<A<AA>>^AvAA^<A>Av<<A>>^AvA^Av<A^>A<Av<A>>^AAvA^Av<A<A>>^AAA<Av>A^A
68 29
980A
^^^A<AvvvA>A
<AAA>Av<<A>>^Av<AAA^>AvA^A
v<<A>>^AAAvA^Av<A<AA>>^AvAA^<A>Av<A<A>>^AAA<Av>A^Av<A^>A<A>A
60 980
179A
^<<A^^A>>AvvvA
<Av<AA>>^A<AA>AvAA^Av<AAA^>A
v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AAvA^Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A
68 179
456A
^^<<A>A>AvvA
<AAv<AA>>^AvA^AvA^Av<AA^>A
v<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>A<A>Av<A^>A<A>Av<A<A>>^AA<Av>A^A
64 456
379A
^A<<^^A>>AvvvA
<A>Av<<AA>^AA>AvAA^Av<AAA^>A
v<<A>>^AvA^Av<A<AA>>^AAvA^<A>AAvA^Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A
64 379
part 1: 126384
part 2: 0


In [4]:
solve_day(input_data)

480A
^^<<A>^AvvvA>A
<AAv<AA>>^AvA^<A>Av<AAA^>AvA^A
v<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>A<Av<A>>^AvA^Av<A<A>>^AAA<Av>A^Av<A^>A<A>A
74 480
143A
^<<A^A>>vAvA
<Av<AA>>^A<A>AvAA<A^>Av<A^>A
v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AvA^Av<A^>AAv<<A>>^A<Av>A^Av<A<A>>^A<Av>A^A
76 143
983A
^^^A<A>vvAvA
<AAA>Av<<A>>^AvA<AA^>Av<A^>A
v<<A>>^AAAvA^Av<A<AA>>^AvAA^<A>Av<A^>Av<<A>>^AA<Av>A^Av<A<A>>^A<Av>A^A
70 983
382A
^A<^^AvvA>vA
<A>Av<<A>^AA>Av<AA^>AvA<A^>A
v<<A>>^AvA^Av<A<AA>>^AvA^<A>AAvA^Av<A<A>>^AA<Av>A^Av<A^>Av<<A>>^A<Av>A^A
72 382
974A
^^^A<<AvA>>vvA
<AAA>Av<<AA>>^Av<A^>AvAA<AA^>A
v<<A>>^AAAvA^Av<A<AA>>^AAvAA^<A>Av<A<A>>^A<Av>A^Av<A^>AAv<<A>>^AA<Av>A^A
72 974
part 1: 212830
part 2: 0


In [5]:
# 212830
# 210338 # too high
# 210000 # too high
# 207406 # nope
# 204914 # nope